In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Function for reading files
def read_csv_with_fallback(filepath, encoding1='utf-16', encoding2='utf-8'):
    try:
        return pd.read_csv(filepath, encoding=encoding1)
    except UnicodeDecodeError:
        try:
            return pd.read_csv(filepath, encoding=encoding2, encoding_errors='replace')
        except UnicodeDecodeError:
            print(f"Failed to read the file: {filepath}")
            return None

In [ ]:
# Try reading the file with utf-16 encoding first
TSG_Children = read_csv_with_fallback("TSG Import/TSGChildren.csv")

In [ ]:
# Try reading the file with utf-16 encoding first
Enrollment_report =read_csv_with_fallback("TSG Import/Enrollmentreport.csv")


In [ ]:
#Report 6510 in CP
TSG_CP_Export = read_csv_with_fallback("TSG Import/TSG6510.csv")

In [ ]:
TSG_CP_Export.shape

In [ ]:
TSG_MasterFile = pd.read_csv("TSG Import/Python TSG_Test_ Master Import Prep File 2025 yr.csv", index_col=None)
TSG_FinalExport= pd.read_csv("TSG Import/TSG final template.csv", index_col=None)

In [ ]:
TSG_MasterFile.head()

In [ ]:
TSG_CP_Export.shape

In [ ]:
TSG_Children_Export = read_csv_with_fallback("TSG Import/TSGChildren.csv")

In [ ]:
original_columns = TSG_MasterFile.columns

# Step 2: Create a new empty DataFrame with the same columns and new row count
TSG_MasterFile = pd.DataFrame(columns=original_columns)
TSG_MasterFile = TSG_MasterFile.reindex(range(len(TSG_CP_Export)))


In [ ]:
#column_names = TSG_CP_Export.columns.tolist()
#print(column_names)

In [ ]:
#column_names_Master = TSG_MasterFile.columns.tolist()
#print(column_names_Master)

In [ ]:
TSG_MasterFile['mm/dd/yy\nbirthdate'] = TSG_CP_Export['birthdate']

In [ ]:
TSG_MasterFile.shape

In [ ]:
TSG_MasterFile['raceID'] = TSG_CP_Export['raceID']
TSG_MasterFile['ethID'] = TSG_CP_Export['ethID']
TSG_MasterFile['LastName'] = TSG_CP_Export['lastName']
TSG_MasterFile['FirstName'] = TSG_CP_Export['firstName']
TSG_MasterFile['genderID'] = TSG_CP_Export['genderID']
TSG_MasterFile['sourceChildID'] = TSG_CP_Export['sourceChildID']
TSG_MasterFile['classID'] = TSG_CP_Export['classID']
TSG_MasterFile['languageID'] = TSG_CP_Export['languageID']
TSG_MasterFile['middleInitial'] = TSG_CP_Export['middleInitial']


In [ ]:
TSG_CP_Export['studentID'] = TSG_CP_Export['identifier'].str.split(':').str[1].str.strip()

# Optionally, convert to numeric if you want to perform further calculations
TSG_CP_Export['studentID'] = pd.to_numeric(TSG_CP_Export['studentID'], errors='coerce')
TSG_CP_Export['studentID'].head()

In [ ]:
TSG_MasterFile['studentID'] = TSG_CP_Export['studentID']

In [ ]:
TSG_MasterFile['studentID'].head()

In [ ]:
TSG_MasterFile.shape

In [ ]:
Enrollment_report['AZED ID'].head()

In [ ]:
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
Enrollment_report['Child ID'] = Enrollment_report['Child ID'].astype(str).str.strip()

In [ ]:
Enrollment_report['Child ID'].head()

In [ ]:
TSG_MasterFile['studentID'].head()

In [ ]:
TSG_MasterFile.to_csv('TSG Import/TSG_MasterFile_test.csv', index=False)

In [ ]:

# Create a dictionary to map 'Child ID' to 'AZED ID' from the Enrollment report
child_id_map = dict(zip(Enrollment_report['Child ID'], Enrollment_report['AZED ID']))

# Use the map to fill the 'correct' column in TSG_MasterFile based on 'studentID' matching 'Child ID'
TSG_MasterFile['correct'] = TSG_MasterFile['studentID'].map(child_id_map).fillna('Not Avail')
TSG_MasterFile['correct'].head()

In [ ]:
TSG_MasterFile['SAISID (For Copy Purpose)']=TSG_MasterFile['correct']

In [ ]:
TSG_MasterFile['SAISID (For Copy Purpose)'].head()

In [ ]:
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
TSG_Children_Export['Student ID'] = TSG_Children_Export['Student ID'].astype(str).str.strip()


In [ ]:
TSG_Children_Export['Identifier'].head()

In [ ]:
TSG_Children_Export[TSG_Children_Export['Student ID'].isna()]

In [ ]:
TSG_Children_Export['Student ID']

In [ ]:
TSG_MasterFile['studentID']

In [ ]:
print(TSG_Children_Export['Student ID'].dtype)
print(TSG_MasterFile['studentID'].dtype)

In [ ]:
# Remove the trailing .0 but keep as string
TSG_Children_Export['Student ID'] = (
    TSG_Children_Export['Student ID']
    .astype(str)                       # make sure it's string
    .str.replace(r'\.0$', '', regex=True)  # strip trailing .0
)

# Also ensure Master column is string (in case it's mixed type)
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()


In [ ]:
TSG_Children_Export['Student ID']

In [ ]:
TSG_MasterFile['studentID']

In [ ]:
child_id_map2 = dict(zip(TSG_Children_Export['Student ID'], TSG_Children_Export['Identifier']))
TSG_MasterFile['TSG Match?'] = TSG_MasterFile['studentID'].map(child_id_map2).fillna('Not Avail')

print(TSG_MasterFile['TSG Match?'].head())

In [ ]:
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
Enrollment_report['Child ID'] = Enrollment_report['Child ID'].astype(str).str.strip()
# Create a dictionary to map 'Child ID' to 'AZED ID' from the Enrollment report
child_id_map = dict(zip(Enrollment_report['Child ID'], Enrollment_report['Enrolled Date']))

# Use the map to fill the 'correct' column in TSG_MasterFile based on 'studentID' matching 'Child ID'
TSG_MasterFile['firstDayInProgram'] = TSG_MasterFile['studentID'].map(child_id_map).fillna('Not Avail')
TSG_MasterFile['firstDayInProgram'].head()

In [ ]:
TSG_MasterFile.columns

In [ ]:
TSG_MasterFile['Do They Match?'] = TSG_MasterFile['correct'] == TSG_MasterFile['TSG Match?']
print(TSG_MasterFile[['correct', 'TSG Match?', 'Do They Match?']].head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up the plot
plt.figure(figsize=(6, 4))
sns.set_style("whitegrid")

# Create the countplot
ax = sns.countplot(x='Do They Match?', data=TSG_MasterFile)

# Annotate the bars with counts
for p in ax.patches:
    count = int(p.get_height())
    ax.annotate(str(count), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom', fontsize=11, color='black')

# Labels and title
plt.title("Validation Match Rate: Do They Match?", fontsize=14)
plt.xlabel("Do They Match?", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
false_matches = TSG_MasterFile[TSG_MasterFile['Do They Match?'] == False]
# Optionally, print the rows where the comparison is False
print(false_matches[['studentID', 'TSG Match?', 'Do They Match?']])

In [ ]:
Enrollment_report.columns

In [ ]:
#change itTSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
Enrollment_report['Child ID'] = Enrollment_report['Child ID'].astype(str).str.strip()

# Create a dictionary to map 'Child ID' to 'AZED ID' from the Enrollment report
child_id_map = dict(zip(Enrollment_report['Child ID'], Enrollment_report['Site Name']))

# Use the map to fill the 'correct' column in TSG_MasterFile based on 'studentID' matching 'Child ID'
TSG_MasterFile["Class name don't copy"] = TSG_MasterFile['studentID'].map(child_id_map).fillna('Not Avail')
TSG_MasterFile["Class name don't copy"].head()

In [ ]:
print(TSG_MasterFile.iloc[:5, 0:19])

In [ ]:
TSG_MasterFile['studentID'] = TSG_MasterFile['studentID'].astype(str).str.strip()
TSG_Children_Export['Student ID'] = TSG_Children_Export['Student ID'].astype(str).str.strip()


child_id_map = dict(zip(TSG_Children_Export['Student ID'], TSG_Children_Export['Child ID']))


TSG_MasterFile['lkupchild id'] = TSG_MasterFile['studentID'].map(child_id_map).fillna('Not Avail')

print(TSG_MasterFile['lkupchild id'].head())

In [ ]:
TSG_MasterFile['ChildID (no formula just for copy purpose)']=TSG_MasterFile['lkupchild id']
TSG_MasterFile['ChildID (no formula just for copy purpose)'].head()

In [ ]:
TSG_MasterFile['CTDSProgramAttendance'] = "072603000"
TSG_MasterFile['CTDSProgramAttendance'] = TSG_MasterFile['CTDSProgramAttendance'].astype(str)
TSG_MasterFile['CTDSProgramAttendance'].head()

In [ ]:
TSG_MasterFile['freeLunch_fl'] = TSG_CP_Export['freeLunch_fl']
TSG_MasterFile['IEPID'] = TSG_CP_Export['IEPID']
TSG_MasterFile['partBEntryDate'] = TSG_CP_Export['partBEntryDate']
TSG_MasterFile['IFSPID'] = TSG_CP_Export['IFSPID']
TSG_MasterFile['partCEntryDate'] = TSG_CP_Export['partCEntryDate']
TSG_MasterFile['colorID'] = TSG_CP_Export['colorID']
TSG_MasterFile['spanishObj_FL'] = TSG_CP_Export['spanishObj_FL']
print(TSG_MasterFile.iloc[:5, 15:28])

In [ ]:
TSG_MasterFile['partBentrytoolid'] = np.where(TSG_MasterFile['IEPID'] == 1, 11, 0)
TSG_MasterFile['partBExitToolID'] = np.where(TSG_MasterFile['IEPID'] == 1, 100, 0)
TSG_MasterFile['partCentrytoolid'] = np.where(TSG_MasterFile['IFSPID'] == 1, 100, 0)

In [ ]:
print(TSG_MasterFile.iloc[:5, 15:32])

In [ ]:
print(TSG_MasterFile.iloc[:5, 0:15])

In [ ]:
true_count = TSG_MasterFile['Do They Match?'].sum()

# Print the result
print(f"The number of True values in 'Do They Match?' column is: {true_count}")

In [ ]:
# Count the number of False values in the 'Do They Match?' column
false_count = (TSG_MasterFile['Do They Match?'] == False).sum()

# Print the result
print(f"The number of False values in 'Do They Match?' column is: {false_count}")

In [ ]:
mask_iep = TSG_MasterFile['IEPID'] == 1


# Create the IEP_Children DataFrame by selecting rows where 'IEPID' equals 1
IEP_Children = TSG_MasterFile.loc[mask_iep].copy()
TSG_MasterFile = TSG_MasterFile.loc[~mask_iep].reset_index(drop=True)
# Optionally, you can remove the rows from TSG_MasterFile if you don't want them in the original DataFrame

IEP_Children.to_csv('TSG Import/IEP_Children.csv', index=False)

In [ ]:
TSG_MasterFile.shape

In [ ]:
IEP_Children = pd.read_csv("TSG Import/IEP_Children.csv", index_col=None)
IEP_Children.shape

In [ ]:
IEP_data = pd.read_csv('TSG Import/IEP school district info.csv')
# Convert the 'IEP/IFSP Date' column to datetime
IEP_data['IEP/IFSP Date'] = pd.to_datetime(IEP_data['IEP/IFSP Date'], format='%m/%d/%Y')
IEP_data.head()
#Sorting the values based on dates
IEP_data_sorted = IEP_data.sort_values(by='IEP/IFSP Date', ascending=False)
# Drop duplicates based on the 'AZED ID' column, keeping the first (most recent) entry
IEP_data_no_duplicates = IEP_data_sorted.drop_duplicates(subset='AZED ID', keep='first')
# Save the cleaned DataFrame to a new CSV file
IEP_data_no_duplicates.to_csv('TSG Import/cleaned_IEPdata.csv', index=False)

print("Duplicates removed and most recent records saved as 'cleaned_IEPdata.csv'")

In [ ]:
lea_mapping = {
'SCHOOL DISTRICT - MESA':	'Mesa Unified',
'AZEIP' :	'Arizona Department of Education as your LEA',
'SCHOOL DISTRICT - TEMPE ELEM':'Tempe School District',
'SCHOOL DISTRICT - CHANDLER' :	'Chandler Unified',
'Apache Junction School District' :	'Apache Junction Unified',
'SCHOOL DISTRICT - GILBERT' :	'Gilbert Unified',
'SCHOOL DISTRICT - KYRENE' :	'Kyrene Elementary',
'Washington Elementary School Dis' :	'Washington Elementary',
'SCHOOL DISTRICT - SCOTTSDALE' :	'Scottsdale Unified',
'Deer Valley Unified School Distr' :	'Deer Valley Unified',
'Nadaburg Unified School District' :	'Nadaburg Unified',
'Avondale Elementary SD #44' :	'Avondale Elementary',
'BUCKEYE ELEMENTARY SCHOOL' : 'Buckeye Elementary',
'Cartwright School district' :	'Cartwright Elementary',
'Balsz Elementary School District' :	'Balsz Elementary',
'Creighton School District' :	'Creighton Elementary',
'Dysart Unified School District' :	'Dysart Unified',
'Litchfield School district' :	'Litchfield Elementary',
'Peoria Unified School District' :	'Peoria Unified',
'ROOSEVELT ELEMENTARY SCHOOL DIST' :	'Roosevelt Elementary',
'ROOSEVELT SCHOOL DISTRICT' :	'Roosevelt Elementary',
'SADDLE MTN UNIFIED SCHOOL DIST' :	'Saddle Mountain Unified',
'TOLLESON SCHOOL DISTRICT' :	'Tolleson Elementary'

}

In [ ]:
cleaned_IEP_data = pd.read_csv('TSG Import/cleaned_IEPdata.csv')
cleaned_IEP_data.columns
cleaned_IEP_data['LEA or Part C Agency'] = cleaned_IEP_data['LEA or Part C Agency'].replace(lea_mapping)
cleaned_IEP_data.head()


In [ ]:
cleaned_IEP_data.columns

In [ ]:
IEP_Children.columns

In [ ]:
IEP_Children['SAISID (For Copy Purpose)'] = (
    IEP_Children['SAISID (For Copy Purpose)']
    .astype(float)  # Convert to float first
    .astype(int)    # Then to int to remove decimal point
    .astype(str)    # Finally to string
    .str.strip()
)

In [ ]:
# Step 1: Force non-numeric to NaN
cleaned_IEP_data['AZED ID'] = pd.to_numeric(cleaned_IEP_data['AZED ID'], errors='coerce')

# Step 2: Drop rows where AZED ID is now NaN
cleaned_IEP_data = cleaned_IEP_data.dropna(subset=['AZED ID'])

# Step 3: Convert cleanly
cleaned_IEP_data['AZED ID'] = (
    cleaned_IEP_data['AZED ID']
    .astype(int)    # now no problem
    .astype(str)    # if you want string for matching
    .str.strip()
)


In [ ]:
cleaned_IEP_data['AZED ID']

In [ ]:
# Create a mapping from 'AZED ID' in cleaned_IEP_data to 'LEA or Part C Agency'
azed_id_map = dict(zip(cleaned_IEP_data['AZED ID'], cleaned_IEP_data['LEA or Part C Agency']))

# Map the 'LEA or Part C Agency' values to 'IEP_Children['CustomResponseValue5 (Copy to import file)']' based on 'SAISID (For Copy Purpose)'
IEP_Children['CustomResponseValue5 (Copy to import file)'] = IEP_Children['SAISID (For Copy Purpose)'] \
    .map(azed_id_map).fillna('Not Avail')
print(IEP_Children['CustomResponseValue5 (Copy to import file)'].head())

In [ ]:
IEP_Children.to_csv('TSG Import/IEP_Children.csv', index=False)
print(IEP_Children[['SAISID (For Copy Purpose)', 'CustomResponseValue5 (Copy to import file)']].head())
IEP_Children.shape

In [ ]:
TSG_MasterFile.to_csv('TSG Import/Final Master File.csv', index=False)

In [ ]:
Template= pd.read_csv("TSG Import/TSG final template.csv", index_col=None)

In [ ]:
Template.shape

In [ ]:
columns_to_copy = Template.columns.tolist()  # Add other columns as needed

# Update the 'first name', 'last name', etc. in abc and xyz based on 'Do They Match?'

#for column in columns_to_copy:
    # Update abc for rows where 'Do They Match?' is True
    #abc[column] = TSG_MasterFile.apply(lambda row: row[column] if row['Do They Match?'] else abc[column], axis=1)
    
    # Update xyz for rows where 'Do They Match?' is False
    #xyz[column] = TSG_MasterFile.apply(lambda row: row[column] if not row['Do They Match?'] else xyz[column], axis=1)

In [ ]:
columns_to_copy

In [ ]:
TSG_MasterFile.columns.tolist()

In [ ]:
NewChildren= pd.read_csv("TSG Import/NewChildren.csv", index_col=None)
print(NewChildren.head())

In [ ]:
print(Template.head())

In [ ]:
NewChildren.columns.tolist()

In [ ]:
column_mapping = {
    'mm/dd/yy\nbirthdate': 'birthdate',
    'raceID': 'raceID',
    'ethID': 'ethID',
    'LastName': 'lastName',
    'FirstName': 'firstName',
    'firstDayInProgram': 'firstDayInProgram',
    'genderID': 'genderID',
    'sourceChildID': 'sourceChildID',
    'classID': 'classID',
    'languageID': 'languageID',
    'middleInitial': 'middleInitial',
    'studentID': 'studentID',
    'SAISID (For Copy Purpose)': 'SAISID',
    'ChildID (no formula just for copy purpose)': 'childID',
    'CTDSProgramAttendance': 'CTDSProgramAttendance',
    'freeLunch_fl': 'freeLunch_fl',
    'IEPID': 'IEPID',
    'partBEntryDate': 'partBEntryDate',
    'partBentrytoolid': 'partBentrytoolid',
    'partBExitToolID': 'partBExitToolID',
    'CustomResponseValue5 (Copy to import file)': 'CustomResponseValue5',
    'IFSPID': 'IFSPID',
    'partCEntryDate': 'partCEntryDate',
    'partCentrytoolid': 'partCentrytoolid',
    'spanishObj_FL': 'spanishObj_FL',
    'colorID': 'colorID'
}

In [ ]:
IEPChildren_readytoimport = pd.read_csv('TSG Import/IEPChildren_readytoimport.csv', index_col=None)
original_columns = IEPChildren_readytoimport.columns

# Step 2: Create a new empty DataFrame with the same columns and new row count
IEPChildren_readytoimport = pd.DataFrame(columns=original_columns)
IEPChildren_readytoimport = IEPChildren_readytoimport.reindex(range(len(IEP_Children)))
ready_to_import = IEPChildren_readytoimport.copy()

# Loop through the column_mapping and copy the corresponding data from IEP_Children to ready_to_import
for old_col, new_col in column_mapping.items():
    if old_col in IEP_Children.columns:
        # Copy the values from the old column in IEP_Children to the new column in ready_to_import
        ready_to_import[new_col] = IEP_Children[old_col]
    else:
        print(f"Warning: Column '{old_col}' not found in IEP_Children.")
ready_to_import['CTDSProgramAttendance'] = "072603000"
# Save the modified DataFrame to a new CSV file
ready_to_import.to_csv('TSG Import/IEPChildren_readytoimport.csv', index=False)


In [ ]:
TSG_MasterFile.columns

In [ ]:

# Ensure 'Do They Match?' is a boolean (True/False)
TSG_MasterFile['Do They Match?'] = TSG_MasterFile['Do They Match?'].astype(bool)
Template = pd.DataFrame(index=range(len(TSG_CP_Export)))
NewChildren = pd.DataFrame(index=range(len(TSG_CP_Export)))
Template = pd.DataFrame(columns=column_mapping.values())
NewChildren = pd.DataFrame(columns=column_mapping.values())


for tsg_master_col, template_col in column_mapping.items():
    # Create masks for True and False conditions
    mask_true = TSG_MasterFile['Do They Match?'] == True
    mask_false = TSG_MasterFile['Do They Match?'] == False
    
       
   
    Template[template_col] = TSG_MasterFile.loc[mask_true, tsg_master_col]
    
  
    NewChildren[template_col] = TSG_MasterFile.loc[mask_false, tsg_master_col]

In [ ]:
NewChildren.head()

In [ ]:
Template.to_csv('TSG Import/Template_ready_to_import.csv', index=False)
NewChildren.to_csv('TSG Import/NewChildren_ready_to_import.csv', index=False)

In [ ]:
Template.shape

In [ ]:
NewChildren.shape